In [1]:
#This notebook builds a heirarchial model from baseline randomforest 
#Morgan fingerprint 
#and cell painting

In [2]:
from rdkit import Chem, DataStructs
from scipy.stats import randint
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns

In [3]:
list_of_lists_df = pd.read_csv("Predictions_train_heldout_scaled_prob_all_assays_ensemble.csv")
list_of_lists_df

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled,simple_pred,Ensemble_Correct
0,588458,"InChI=1S/C19H19N3O5S3/c23-16-6-5-13(30(25,26)2...",0.072320,0,0.227732,0.291514,1,0.291203,0.0,Training,True,False,0.158783,0.500219,0.0,True
1,588458,InChI=1S/C9H8ClN3S/c1-6-11-12-9(14)13(6)8-4-2-...,0.241077,1,0.227732,0.334301,1,0.291203,0.0,Training,False,False,0.508641,0.530402,1.0,False
2,588458,InChI=1S/C20H20N2O4/c1-2-25-20(24)17-12-18-16(...,0.119513,0,0.227732,0.186209,0,0.291203,0.0,Training,True,True,0.262399,0.319723,0.0,True
3,588458,InChI=1S/C26H24N2O5S/c1-33-19-11-9-18(10-12-19...,0.045993,0,0.227732,0.472894,1,0.291203,1.0,Training,False,True,0.100980,0.628169,0.0,False
4,588458,InChI=1S/C19H21N3O3S/c1-13-10-16-11-15(4-9-19(...,0.269438,1,0.227732,0.168755,0,0.291203,0.0,Training,False,True,0.527003,0.289754,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50191,1117305,"InChI=1S/C7H8ClN3O4S2/c8-4-1-5-7(2-6(4)16(9,12...",0.703995,1,0.457779,0.556806,1,0.264151,0.0,HeldOut,False,False,0.727044,0.698855,1.0,False
50192,1117305,InChI=1S/C14H9I3O4/c15-9-6-8(1-2-12(9)18)21-14...,0.196017,0,0.457779,0.210436,0,0.264151,0.0,HeldOut,True,True,0.214095,0.398324,0.0,True
50193,1117305,InChI=1S/C18H19Cl2NO4/c1-5-25-18(23)14-10(3)21...,0.202912,0,0.457779,0.487726,1,0.264151,1.0,HeldOut,False,True,0.221627,0.651916,0.0,False
50194,1117305,InChI=1S/C16H11BrN2O/c17-9-5-6-14-11(7-9)12-8-...,0.149597,0,0.457779,0.238125,0,0.264151,0.0,HeldOut,True,True,0.163395,0.450736,0.0,True


In [4]:
from sklearn.linear_model import LogisticRegression
from io import StringIO
from itertools import product
import sys
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold

In [5]:
def RF_Fn(assay):

    detail_list=pd.DataFrame()
    detail_list = list_of_lists_df[list_of_lists_df["assay"]==assay].reset_index(drop=True)
    
    
    proba_batch=[]
    pred_batch=[]
    true_batch=[]
    StdInChI_batch=[]
    
    train= detail_list[detail_list["Data"]=="Training"]

    test= detail_list[detail_list["Data"]=="HeldOut"]
    

    print("Herirachal model for training data")    

    print("RF Model")
            
    X_train = train[["fp_proba_scaled", "CP_proba_scaled"]].to_numpy()
    y_train = train["true"].to_numpy()
    X_test = test[["fp_proba_scaled", "CP_proba_scaled"]].to_numpy()
    y_test = test["true"].to_numpy()


    inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
         
    param_dist_grid = {
        'n_estimators':[100, 300, 400, 500],
        'random_state': [42],
        'n_jobs': [1],
        'class_weight' : [None, 'balanced']
        }
        
    rf = RandomForestClassifier(n_jobs=-1)
    rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, 
                                random_state=42, n_jobs=20, verbose=0,  cv = inner_cv)

    rsh.fit(X_train, y_train)

    print("Tuned Mode: ", rsh.best_params_)
    clf = rsh.best_estimator_
        
    #n_estimators=100
    #clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=10, random_state=42, class_weight="balanced")           
    clf.fit(X_train, y_train)
    
    #Threshold Balancing
    cross_val_prob = cross_val_predict(rsh.best_estimator_, X_train, y_train, cv=inner_cv, method='predict_proba')[:, 1]
    # calculate roc curves
    fpr, tpr, thresholds = roc_curve(y_train, cross_val_prob)
    # get the best threshold
    J = tpr - fpr
    ix = argmax(J)
    best_thresh = thresholds[ix]
    print('Best Threshold=%f' % (best_thresh))


    proba = clf.predict_proba(X_test)[:,1]
    pred  = [ 1 if x>best_thresh  else 0 for x in proba ] 

           
    StdInChI = test["InChICode_standardised"].to_numpy()
    #pred = clf.predict(X_test) 
    #proba = clf.predict_proba(X_test)[:,1]
    true = test["true"].to_numpy()
            

    StdInChI_batch = np.concatenate([StdInChI_batch, StdInChI])
    proba_batch = np.concatenate([proba_batch, proba])
    pred_batch = np.concatenate([pred_batch, pred])
    true_batch = np.concatenate([true_batch, true])

    print(len(pred_batch))
    print(len(true_batch))
   
    return StdInChI_batch, true_batch, pred_batch, proba_batch

In [6]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score

def check_heirarchial(assay, method, y_true, y_pred, y_prob):
    
    conf_matrix = confusion_matrix(y_true, y_pred)
    #print(conf_matrix)
    
    #print(classification_report(y_true, y_pred))
    
    ba= balanced_accuracy_score(y_true, y_pred)
    #print('balanced_accuracy_score ',ba)
    
    mcc=matthews_corrcoef(y_true, y_pred)
    #print('matthews_corrcoef ',mcc)
    # Sensitivity, hit rate, recall, or true positive rate
    
    f1= f1_score(y_true, y_pred, average='binary')
    precision = precision_score(y_true, y_pred, average='binary')
    
    #print('F1 Toxic', f1)
    #print('precision', precision)
    
    Specificity = conf_matrix[0,0]/(conf_matrix[0,0]+conf_matrix[0,1])
    # Specificity or true negative rate
    Sensitivity = conf_matrix[1,1]/(conf_matrix[1,0]+conf_matrix[1,1]) 
    
    #print( 'Sensitivity', Sensitivity)
    #print( 'Specificity', Specificity)
    AUC = roc_auc_score(y_true, y_prob)
    #print('AUC-ROC ',AUC)    
    # calculate roc curves
    best_model_fpr, best_model_tpr, _ = roc_curve(y_true, y_prob)   
    
    
    row=[assay, method, f1, precision, Sensitivity, Specificity, ba, mcc, AUC]
    metrics_list.append(row)
    
    return metrics_list

In [7]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from collections import Counter
from numpy import argmax
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

In [8]:
import warnings
warnings.filterwarnings("ignore") #ignore, default
import pandas

#sns.set(rc={'figure.figsize':(2, 2)})
#plt.rcParams['figure.dpi'] = 80
#sns.set_style("white")
metrics_list=[]
all_preds_test_compounds=pd.DataFrame(columns=['assay', 'InChICode_standardised', 'fp_proba', 'fp_pred', 'fp_threshold', 'CP_proba',
       'CP_pred', 'CP_threshold', 'true', 'Data', 'MFP_Correct',
       'CP_Correct', 'fp_proba_scaled', 'CP_proba_scaled', 'simple_pred',
       'Ensemble_Correct', 'heirarchial_pred', 'heirarchial_Correct'])


#for assay in tqdm(assaylist[:7]):
for assay in tqdm(list_of_lists_df.assay.unique()[:]):
    
    print("Assay: ", assay)
    detail_list=pd.DataFrame()
    detail_list = list_of_lists_df[list_of_lists_df["assay"]==assay].reset_index(drop=True)
    detail_list_test = detail_list[detail_list["Data"]=="HeldOut"]

    print("Heirarchial_Model")
    merger_df=pd.DataFrame()
    
    StdInChI_batch, true_batch, pred_batch, proba_batch = RF_Fn(assay)
    check_heirarchial(assay, "Heirarchial Model", true_batch, pred_batch, proba_batch)
    
    #print(true_batch)
    
    print(assay)
    
    merger_df["InChICode_standardised"]=StdInChI_batch
    merger_df["heirarchial_pred"]= pred_batch
    merger_df["true"]=true_batch
    merger_df["heirarchial_Correct"]= merger_df["heirarchial_pred"] == merger_df["true"]
    merger_df["assay"]=assay
    
    
    #For each assay combine original predictions and merger_df predictions
    #all_preds = pd.DataFrame()
    all_preds_test_compounds= pd.concat([all_preds_test_compounds, pd.merge(detail_list_test, merger_df)])  
    
    
    

  0%|                                                    | 0/88 [00:00<?, ?it/s]

Assay:  588458
Heirarchial_Model
Herirachal model for training data
RF Model
Tuned Mode:  {'random_state': 42, 'n_jobs': 1, 'n_estimators': 400, 'class_weight': 'balanced'}


  1%|▌                                           | 1/88 [00:06<08:52,  6.12s/it]

Best Threshold=0.565000
90
90
588458
Assay:  588334
Heirarchial_Model
Herirachal model for training data
RF Model
Tuned Mode:  {'random_state': 42, 'n_jobs': 1, 'n_estimators': 500, 'class_weight': 'balanced'}


  2%|█                                           | 2/88 [00:12<08:44,  6.10s/it]

...

Best Threshold=0.186000
176
176
1159524
Assay:  1117304
Heirarchial_Model
Herirachal model for training data
RF Model
Tuned Mode:  {'random_state': 42, 'n_jobs': 1, 'n_estimators': 400, 'class_weight': 'balanced'}


 99%|██████████████████████████████████████████▌| 87/88 [07:07<00:04,  4.56s/it]

Best Threshold=0.210000
78
78
1117304
Assay:  1117305
Heirarchial_Model
Herirachal model for training data
RF Model
Tuned Mode:  {'random_state': 42, 'n_jobs': 1, 'n_estimators': 500, 'class_weight': 'balanced'}


100%|███████████████████████████████████████████| 88/88 [07:13<00:00,  4.93s/it]

Best Threshold=0.584000
72
72
1117305


In [9]:
all_preds_test_compounds

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled,simple_pred,Ensemble_Correct,heirarchial_pred,heirarchial_Correct
0,588458,InChI=1S/C14H15BrN2O3/c1-17(2)6-5-16-13(18)11-...,0.335765,1,0.310223,0.478519,1,0.304308,0.0,HeldOut,False,False,0.518514,0.625207,1.0,False,0.0,True
1,588458,InChI=1S/C23H22N2O2/c26-23(16-22-21-9-5-4-6-17...,0.210945,0,0.310223,0.168993,0,0.304308,0.0,HeldOut,True,True,0.339990,0.277667,0.0,True,0.0,True
2,588458,InChI=1S/C22H17F2NO3/c23-18-7-1-15(2-8-18)13-2...,0.155810,0,0.310223,0.248408,0,0.304308,0.0,HeldOut,True,True,0.251126,0.408151,0.0,True,0.0,True
3,588458,InChI=1S/C24H24N6O4/c25-22-20(24(31)26-7-8-29-...,0.163149,0,0.310223,0.208577,0,0.304308,0.0,HeldOut,True,True,0.262955,0.342707,0.0,True,0.0,True
4,588458,InChI=1S/C15H23BrN2O4S/c1-10-8-18(11(2)9-19)23...,0.285539,0,0.310223,0.554642,1,0.304308,0.0,HeldOut,True,False,0.460216,0.679917,1.0,False,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,1117305,"InChI=1S/C7H8ClN3O4S2/c8-4-1-5-7(2-6(4)16(9,12...",0.703995,1,0.457779,0.556806,1,0.264151,0.0,HeldOut,False,False,0.727044,0.698855,1.0,False,0.0,True
68,1117305,InChI=1S/C14H9I3O4/c15-9-6-8(1-2-12(9)18)21-14...,0.196017,0,0.457779,0.210436,0,0.264151,0.0,HeldOut,True,True,0.214095,0.398324,0.0,True,0.0,True
69,1117305,InChI=1S/C18H19Cl2NO4/c1-5-25-18(23)14-10(3)21...,0.202912,0,0.457779,0.487726,1,0.264151,1.0,HeldOut,False,True,0.221627,0.651916,0.0,False,0.0,False
70,1117305,InChI=1S/C16H11BrN2O/c17-9-5-6-14-11(7-9)12-8-...,0.149597,0,0.457779,0.238125,0,0.264151,0.0,HeldOut,True,True,0.163395,0.450736,0.0,True,0.0,True


In [10]:
all_preds_test_compounds.to_csv("Predictions_train_heldout_scaled_prob_all_assays_heirarchial_model_test_compounds.csv", index=False)

In [11]:
metrics_list = pd.DataFrame(metrics_list,columns=["assay","method","f1", "precision", "Sensitivity", "Specificity", "ba", "mcc", "AUC"])
metrics_list

,assay,method,f1,precision,Sensitivity,Specificity,ba,mcc,AUC
0,588458,Heirarchial Model,0.312500,0.500000,0.227273,0.926471,0.576872,0.210241,0.664104
1,588334,Heirarchial Model,0.320988,0.236364,0.500000,0.475000,0.487500,-0.021528,0.533413
2,2642,Heirarchial Model,0.280000,0.225806,0.368421,0.582609,0.475515,-0.043098,0.470252
3,2156,Heirarchial Model,0.408759,0.256881,1.000000,0.058140,0.529070,0.122209,0.596761
4,2330,Heirarchial Model,0.750000,0.750000,0.750000,0.920000,0.835000,0.670000,0.894375
...,...,...,...,...,...,...,...,...,...
83,720504,Heirarchial Model,0.863636,0.760000,1.000000,0.142857,0.571429,0.329502,0.293233
84,720532,Heirarchial Model,0.507937,0.800000,0.372093,0.777778,0.574935,0.145606,0.516150
85,1159524,Heirarchial Model,0.375000,0.270000,0.613636,0.446970,0.530303,0.052981,0.530045
86,1117304,Heirarchial Model,0.525000,0.381818,0.840000,0.358491,0.599245,0.203143,0.579245


In [12]:
metrics_list.to_csv("heirarchial_model_metrics.csv", index=False)